# Truss Analyer Testing Zone

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import pandas as pd
import re
import math
from PyNite import FEModel3D
from PyNite.Visualization import render_model
from rich import print
import pandas as pd
import matplotlib.pyplot as plt
# import truss_input as ti
import truss_model as tm

In [ ]:
top_nodes = []
bot_nodes = []
d_w = L/(n/2)
for i, loc in enumerate(list(np.linspace(0, L, int(n/2+1)))):
    t_node = [loc, d]
    top_nodes.append(t_node)
    if i == 0:
        continue
    else: 
        b_node = [loc-d_w/2, 0] 
        bot_nodes.append(b_node)

In [4]:
# Import Canam's tables

def remove_comma(cell):
    number = float(cell.replace(',', ''))
    return number

canam_df = pd.read_csv('OWSJ_tables_canam.csv',header=1)
canam_df.rename(columns={canam_df.columns[0]: 'span', canam_df.columns[1]: 'depth'}, inplace=True)
canam_df['span'].ffill(inplace=True)
canam_df['span']= canam_df['span']*1000
canam_df['depth'] = canam_df['depth'].apply(remove_comma)
canam_df['depth'] = canam_df['depth'].astype(float)
canam_df.head(10)

# save as json for linux reading
# canam_df.to_json('canam_df.json', orient='records')

/tmp/ipykernel_3737/2775499420.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  canam_df['span'].ffill(inplace=True)


,span,depth,4.5,6.0,7.5,9.0,10.5,12.0,13.5,15.0,16.5,18.0,19.5,21.0,22.5
0,3000.0,200.0,9.0\n6.0\n1.03,9.0\n7.7\n1.03,9.0\n7.7\n1.03,9.0\n7.7\n1.01,9.3\n9.6\n1.01,9.3\n9.6\n1.01,9.7\n10.1\n1.02,9.7\n10.1\n1.02,9.7\n10.1\n1.01,9.8\n10.3\n1.03,10.1\n10.5\n1.03,10.4\n10.9\n1.05,12.0\n12.9\n1.09
1,3000.0,250.0,9.1\n6.0\n1.03,9.1\n8.0\n1.03,9.1\n10.0\n1.03,9.1\n11.5\n1.01,9.5\n14.0\n1.02,9.5\n14.1\n1.02,9.9\n15.0\n1.02,9.9\n15.0\n1.02,9.9\n15.0\n1.01,9.9\n15.0\n1.01,10.3\n15.7\n1.02,10.3\n15.7\n1.02,10.3\n15.7\n1.03
2,3000.0,300.0,9.0\n6.0\n1.00,9.0\n8.0\n1.00,9.0\n10.0\n1.00,9.6\n12.0\n1.00,9.6\n14.0\n1.00,9.6\n16.0\n1.00,9.8\n18.0\n1.00,9.8\n19.9\n1.00,10.1\n20.2\n1.00,10.1\n20.3\n1.00,10.2\n21.3\n1.00,10.2\n21.3\n1.00,10.2\n21.3\n1.00
3,3000.0,350.0,9.1\n6.0\n1.00,9.1\n8.0\n1.00,9.1\n10.0\n1.00,9.8\n12.0\n1.03,10.0\n14.0\n1.01,10.0\n16.0\n1.01,10.0\n18.0\n1.00,10.0\n20.0\n1.00,10.5\n22.0\n1.01,10.5\n24.0\n1.01,10.5\n26.0\n1.00,10.5\n27.3 1.00,11.0\n28.9\n1.01
4,3000.0,400.0,12.7\n6.0\n1.25,12.7\n8.0\n1.25,12.7\n10.0\n1.25,12.7\n12.0\n1.22,12.7\n14.0\n1.19,12.7\n16.0\n1.19,12.7\n18.0\n1.19,12.7\n20.0\n1.19,12.7\n22.0\n1.17,12.7\n24.0\n1.17,12.7\n26.0\n1.16,12.7\n28.0\n1.17,12.7\n30.0\n1.16
5,3000.0,450.0,12.9\n6.0\n1.26,12.9\n8.0\n1.26,12.9\n10.0\n1.26,12.9\n12.0\n1.23,12.9\n14.0\n1.20,12.9\n16.0\n1.20,12.9\n18.0\n1.19,12.9\n20.0\n1.19,12.9\n22.0\n1.18,12.9\n24.0\n1.18,12.9\n26.0\n1.17,12.9\n28.0\n1.17,12.9\n30.0\n1.16
6,3000.0,500.0,13.1\n6.0\n1.26,13.1\n8.0\n1.26,13.1\n10.0\n1.26,13.1\n12.0\n1.23,13.1\n14.0\n1.20,13.1\n16.0\n1.20,13.1\n18.0\n1.20,13.1\n20.0\n1.20,13.1\n22.0\n1.18,13.1\n24.0\n1.18,13.1\n26.0\n1.17,13.1\n28.0\n1.17,13.1\n30.0\n1.16
7,4000.0,200.0,8.8\n4.0\n1.03,8.8\n4.0\n1.01,8.9\n4.2\n1.01,9.4\n4.6\n1.02,9.4\n4.6\n1.02,11.5\n5.4\n1.06,12.2\n5.8\n1.05,12.1\n6.2\n1.09,13.4\n6.9\n1.12,14.3\n7.5\n1.12,15.2\n8.1\n1.08,17.0\n9.1\n1.13,17.8\n9.5\n1.16
8,4000.0,250.0,8.9\n6.0\n1.03,8.9\n6.1\n1.01,8.9\n6.0\n1.01,9.6\n7.0\n1.02,9.6\n7.0\n1.01,10.0\n7.2\n1.01,10.0\n7.2\n1.01,11.4\n8.3\n1.05,12.9\n9.0\n1.04,12.5\n9.3\n1.04,12.9\n9.7\n1.01,13.7\n10.5\n1.01,14.9\n11.3\n1.01
9,4000.0,300.0,8.9\n6.0\n1.00,9.1\n8.0\n1.00,9.1\n8.0\n1.00,10.2\n8.7\n1.00,10.4\n9.3\n1.00,10.6\n9.4\n1.00,11.2\n10.1\n1.00,11.7\n10.3\n1.01,12.6\n11.0\n1.01,13.3\n11.8\n1.03,14.7\n13.3\n1.03,15.0\n13.8\n1.00,16.1\n14.7\n1.03


In [98]:
# Import Vulcraft's data
vulcraft_df = pd.read_csv('OWSJ_tables_vulcraft.csv', header=1)
vulcraft_df.rename(columns={"Factored Load": 'value'}, inplace=True)
vulcraft_df.columns = [col.lower() for col in vulcraft_df]
#drop first row and all bridging rows
vulcraft_df.drop(index=0, inplace=True)
vulcraft_df = vulcraft_df[~vulcraft_df['value'].str.contains('Bridg.')]
#fill depths
vulcraft_df['depth'].bfill(inplace=True)
#convert all span values to mm
vulcraft_df.span = vulcraft_df.span.astype(float)
vulcraft_df.span = vulcraft_df.span*1000
vulcraft_df.head(30)

# save as json for linux reading
# vulcraft_df.to_json('vulcraft_df.json', orient='records')

In [41]:
# Import Omega's tables

def find_span(cell):
    if "SPAN" in cell:
        match_span = re.search(r'\b(\d+)\b', cell)
        return match_span.group(1)
    else:
        return np.nan

omega_df = pd.read_csv('OWSJ_tables_omega.csv', header=None)
col_names = ["depth", "D/D", "E/D", "E/E", "F/E", "F/F", "G/F", "G/G", "H/G", "H/H", "K/H", "K/K", "L/K", "L/L"]
omega_df.columns = col_names
omega_df['span'] = omega_df['depth'].apply(find_span)
omega_df['span'].ffill(inplace=True)
columns = ['span'] + [col for col in omega_df if col != 'span']
omega_df = omega_df[columns]
# Drop rows with useless info; cells with "DEPTH" OR "SPAN"
omega_df = omega_df[~omega_df['depth'].str.contains('DEPTH') & ~omega_df['depth'].str.contains('SPAN')]

# save as json for linux reading
# omega_df.to_json('omega_df.json', orient='records')

/tmp/ipykernel_3737/2327350521.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  omega_df['span'].ffill(inplace=True)


In [43]:
import tabula
tables = tabula.read_pdf("metric tables from Vulcraft OWSJ.pdf", pages='all', multiple_tables=True, pandas_options={'header': None})

In [84]:
# Start working with the saved dataframes
canam_df = pd.read_json("canam_df.json")
omega_df = pd.read_json("omega_df.json")
vulcraft_df = pd.read_json("vulcraft_df.json")
canam_df.name = "canam"
omega_df.name = "omega"
vulcraft_df.name = "vulcraft"

OWSJ_dfs = [canam_df, omega_df, vulcraft_df]

span = 17565
depth = 933
f_load = 8

In [102]:

# load_mask = (canam_df['depth'] < 1.05*depth) & (canam_df['depth'] > 0.95*depth) 

for df in OWSJ_dfs:
    name = df.name
    print(name)
    span_mask = (df['span'] < 1.05*span) & (df['span'] > 0.95*span) 
    depth_mask = (df['depth'] < 1.05*depth) & (df['depth'] > 0.95*depth)
    df = df.loc[span_mask & depth_mask]
    #special case for Omega
    if name == "omega":
        cols = [col for col in omega_df.columns if '/' in col]
        df_sliced = df[cols]
        chords = []
        for column in df_sliced.columns:
            for index, cell_value in enumerate(df_sliced[column]):
                try:
                    f = float(cell_value.split("\n")[0])
                    sw = float(cell_value.split("\n")[2])
                    if (f > 0.98*f_load) & (f < 1.02*f_load):
                        chords.append((column, sw))
                except:
                    continue
              
    elif name == "vulcraft":
        df = df[df['value'].str.contains('Self')] #drop L/360 row
        col_nums = [float(col) for col in df.columns if "." in col] #convert factored headers to float
        sorted_numbers = np.sort(col_nums) 
        index = np.searchsorted(sorted_numbers, f_load, side='right') - 1 #use np method to determine number closest, but not over factored load
        sw_vulcraft = df[str(sorted_numbers[index])] #convert the found number back to string as save a df of sw

    elif name == "canam":
        col_nums = [float(col) for col in df.columns if "." in col] #convert factored headers to float
        sorted_numbers = np.sort(col_nums) 
        index = np.searchsorted(sorted_numbers, f_load, side='right') - 1 #use np method to determine number closest, but not over factored load
        sw_canam = df[str(sorted_numbers[index])] #convert the found number back to string as save a df of sw




canam

omega

vulcraft

In [103]:
sw_canam

99     19.1\n2.6\n1.00
105    22.0\n2.6\n1.00
Name: 7.5, dtype: object

In [94]:
sw_vulcraft

258    18.4
274    20.7
Name: 6.3, dtype: float64

In [90]:
chords

[('H/G', 19.31),
 ('H/H', 19.89),
 ('K/H', 20.33),
 ('K/H', 20.46),
 ('K/K', 20.77),
 ('K/K', 21.15),
 ('K/K', 21.26),
 ('L/K', 21.48),
 ('L/K', 21.67),
 ('L/K', 21.97)]

In [96]:
canam_df.head(10)

,span,depth,4.5,6.0,7.5,9.0,10.5,12.0,13.5,15.0,16.5,18.0,19.5,21.0,22.5
0,3000,200,9.0\n6.0\n1.03,9.0\n7.7\n1.03,9.0\n7.7\n1.03,9.0\n7.7\n1.01,9.3\n9.6\n1.01,9.3\n9.6\n1.01,9.7\n10.1\n1.02,9.7\n10.1\n1.02,9.7\n10.1\n1.01,9.8\n10.3\n1.03,10.1\n10.5\n1.03,10.4\n10.9\n1.05,12.0\n12.9\n1.09
1,3000,250,9.1\n6.0\n1.03,9.1\n8.0\n1.03,9.1\n10.0\n1.03,9.1\n11.5\n1.01,9.5\n14.0\n1.02,9.5\n14.1\n1.02,9.9\n15.0\n1.02,9.9\n15.0\n1.02,9.9\n15.0\n1.01,9.9\n15.0\n1.01,10.3\n15.7\n1.02,10.3\n15.7\n1.02,10.3\n15.7\n1.03
2,3000,300,9.0\n6.0\n1.00,9.0\n8.0\n1.00,9.0\n10.0\n1.00,9.6\n12.0\n1.00,9.6\n14.0\n1.00,9.6\n16.0\n1.00,9.8\n18.0\n1.00,9.8\n19.9\n1.00,10.1\n20.2\n1.00,10.1\n20.3\n1.00,10.2\n21.3\n1.00,10.2\n21.3\n1.00,10.2\n21.3\n1.00
3,3000,350,9.1\n6.0\n1.00,9.1\n8.0\n1.00,9.1\n10.0\n1.00,9.8\n12.0\n1.03,10.0\n14.0\n1.01,10.0\n16.0\n1.01,10.0\n18.0\n1.00,10.0\n20.0\n1.00,10.5\n22.0\n1.01,10.5\n24.0\n1.01,10.5\n26.0\n1.00,10.5\n27.3 1.00,11.0\n28.9\n1.01
4,3000,400,12.7\n6.0\n1.25,12.7\n8.0\n1.25,12.7\n10.0\n1.25,12.7\n12.0\n1.22,12.7\n14.0\n1.19,12.7\n16.0\n1.19,12.7\n18.0\n1.19,12.7\n20.0\n1.19,12.7\n22.0\n1.17,12.7\n24.0\n1.17,12.7\n26.0\n1.16,12.7\n28.0\n1.17,12.7\n30.0\n1.16
5,3000,450,12.9\n6.0\n1.26,12.9\n8.0\n1.26,12.9\n10.0\n1.26,12.9\n12.0\n1.23,12.9\n14.0\n1.20,12.9\n16.0\n1.20,12.9\n18.0\n1.19,12.9\n20.0\n1.19,12.9\n22.0\n1.18,12.9\n24.0\n1.18,12.9\n26.0\n1.17,12.9\n28.0\n1.17,12.9\n30.0\n1.16
6,3000,500,13.1\n6.0\n1.26,13.1\n8.0\n1.26,13.1\n10.0\n1.26,13.1\n12.0\n1.23,13.1\n14.0\n1.20,13.1\n16.0\n1.20,13.1\n18.0\n1.20,13.1\n20.0\n1.20,13.1\n22.0\n1.18,13.1\n24.0\n1.18,13.1\n26.0\n1.17,13.1\n28.0\n1.17,13.1\n30.0\n1.16
7,4000,200,8.8\n4.0\n1.03,8.8\n4.0\n1.01,8.9\n4.2\n1.01,9.4\n4.6\n1.02,9.4\n4.6\n1.02,11.5\n5.4\n1.06,12.2\n5.8\n1.05,12.1\n6.2\n1.09,13.4\n6.9\n1.12,14.3\n7.5\n1.12,15.2\n8.1\n1.08,17.0\n9.1\n1.13,17.8\n9.5\n1.16
8,4000,250,8.9\n6.0\n1.03,8.9\n6.1\n1.01,8.9\n6.0\n1.01,9.6\n7.0\n1.02,9.6\n7.0\n1.01,10.0\n7.2\n1.01,10.0\n7.2\n1.01,11.4\n8.3\n1.05,12.9\n9.0\n1.04,12.5\n9.3\n1.04,12.9\n9.7\n1.01,13.7\n10.5\n1.01,14.9\n11.3\n1.01
9,4000,300,8.9\n6.0\n1.00,9.1\n8.0\n1.00,9.1\n8.0\n1.00,10.2\n8.7\n1.00,10.4\n9.3\n1.00,10.6\n9.4\n1.00,11.2\n10.1\n1.00,11.7\n10.3\n1.01,12.6\n11.0\n1.01,13.3\n11.8\n1.03,14.7\n13.3\n1.03,15.0\n13.8\n1.00,16.1\n14.7\n1.03


In [24]:
span_mask = (canam_df['span'] < 1.05*span) & (canam_df['span'] > 0.95*span) 
depth_mask = (canam_df['depth'] < 1.05*depth) & (canam_df['depth'] > 0.95*depth) 
load_mask = (canam_df['depth'] < 1.05*depth) & (canam_df['depth'] > 0.95*depth) 

In [25]:
canam_df.loc[span_mask & depth_mask]

,span,depth,4.5,6.0,7.5,9.0,10.5,12.0,13.5,15.0,16.5,18.0,19.5,21.0,22.5
99,17000,900,15.1\n1.9\n1.00,17.4\n2.3\n1.00,19.1\n2.6\n1.00,21.6\n3.0\n1.00,24.4\n3.3\n1.00,26.5\n3.7\n1.00,29.4\n4.1\n1.00,31.5\n4.4\n1.00,34.3\n4.8\n1.00,36.2\n5.1\n1.13,38.3\n5.4\n1.15,41.7\n5.9\n1.14,46.0\n6.7\n1.18
105,18000,900,16.3\n1.8\n1.00,19.3\n2.2\n1.00,22.0\n2.6\n1.00,24.8\n2.9\n1.00,28.0\n3.3\n1.00,30.5\n3.6\n1.00,33.4\n4.0\n1.00,37.5\n4.5\n1.03,38.8\n4.8\n1.17,44.9\n5.5\n1.21,45.7\n5.7\n1.17,49.5\n6.1\n1.24,52.9\n6.6\n1.26


In [81]:
# span_mask = (omega_df['span'] < 1.05*span) & (omega_df['span'] > 0.95*span) 
# depth_mask = (omega_df['depth'] < 1.05*depth) & (omega_df['depth'] > 0.95*depth) 
# masked_df = omega_df.loc[span_mask & depth_mask]
cols = [col for col in omega_df.columns if '/' in col]
df_sliced = omega_df[cols]


for column in df_sliced.columns:
    # print(column)
    for index, cell_value in enumerate(df_sliced[column]):
        try:
            f = float(cell_value.split("\n")[0])
            if (f > 0.95*f_load) & (f < 1.05*f_load):
                print(column)
                print(f)
        except:
            print(cell_value, index)
            

# for index, row in df_sliced.iterrows():
#     print(row[0])
    # for value in row:
    #     f = float(value.split("\n")[0])
    #     if (f > 0.95*f_load) & (f < 1.05*f_load):
    #         print(row)
    #         print(index)
    #         print(f)
    #         print(omega_df.columns[index])

D/D

8.05

D/D

8.0

D/D

...

8.04

L/L

8.07

L/L

8.04

In [32]:
span_mask = (vulcraft_df['span'] < 1.05*span) & (vulcraft_df['span'] > 0.95*span) 
depth_mask = (vulcraft_df['depth'] < 1.05*depth) & (vulcraft_df['depth'] > 0.95*depth) 
vulcraft_df.loc[span_mask & depth_mask]

,span,depth,value,4.5,6.3,8.1,9.9,11.7,13.5,15.3,17.1,18.9,20.7,22.5,24.3
258,17000,914,Self Wt (kg/m),15.9,18.4,21.0,25.7,26.3,29.0,30.8,34.0,37.9,40.0,45.3,46.0
259,17000,914,WL/360 (kN/m),2.0,2.0,3.0,4.0,4.0,4.0,5.0,5.0,6.0,6.0,7.0,7.0
274,18000,914,Self Wt (kg/m),16.1,20.7,20.9,26.5,28.6,31.0,34.4,38.1,41.7,46.2,51.5,53.2
275,18000,914,WL/360 (kN/m),2.0,3.0,3.0,3.0,4.0,4.0,5.0,5.0,6.0,6.0,6.0,7.0
